In [1]:
from bs4 import BeautifulSoup
from time import sleep
import json
import sys
from urllib.request import urlopen
import string
sys.setrecursionlimit(100000)

In [15]:
# Current number of pages of Borowitz Reports
num_pages = 94

# First page
base_url = 'https://www.newyorker.com/humor/borowitz-report'

# All subsequent pages
base_url_page = 'https://www.newyorker.com/humor/borowitz-report/page/'
urls = [base_url_page + str(i) for i in range(2,num_pages+1)]

# Here is our list of page URLs
page_urls = [base_url]
page_urls.extend(urls)

In [16]:
# This block of code scrapes the unique URL portion specific to each article on each page and stores them in links
links = []
print('Scraping all article link addresses')
for url in page_urls:
    sleep(0.1)
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    #stuff = soup.find_all("a", attrs={"class":"Link__link___3dWao  "})
    stuff2 = soup.find_all('li', attrs={"class":"River__riverItem___3huWr"})
    things = []
    for blah in stuff2:
        if blah.find_next("h4"):
            things.append(blah.find_all("a", attrs={"class":"Link__link___3dWao"}))

    for thing in things:
        links.append(thing[0].findNext({"a":"href"}).get('href'))

print('Done.')

Scraping all article link addresses
Done.


In [2]:
# Clean up the article text for each article

def article_clean(article):
    article = article.replace('\xa0', u' ')
    article = article.replace("Get news satire from The Borowitz Report delivered to your inbox.", "")
    article = article.replace("Get the Borowitz Report delivered to your inbox.", "")
    article = article.replace("Get The Borowitz Report delivered to your inbox.", "")
    article = article.replace("(The Borowitz Report)", "")
    article = article.replace("News Satire from The Borowitz Report", "")
    article = article.replace("Satire from The Borowitz Report", "")
    article = article.replace("This post is news satire from The Borowitz Report.", "")
    article = article.replace("News satire from The Borowitz Report.", "")
    article = article.replace("Andy Borowitz will be doing a free show at Rutgers University on Monday, October 29, at 7 P.M. To register for tickets, click here.    Photograph by Lauren Lancaster.", "")
    article = article.replace("Andy Borowitz is doing a show to benefit public radio.","")
    article = article.replace("Tickets for Andy Borowitz's next live show are now on sale.", "")
    article = article.replace("(Satire from The Borowitz Report)", "")
    article = article.replace("Illustration by Andy Borowitz.", "")
    article = article.replace("Andy Borowitz will be doing two shows at next month's New Yorker Festival: Friday, October 5th, with the storytelling group The Moth, and Saturday, October 6th, with Sarah Silverman. Ticket information here.", "")
    article = article.replace("A small number of tickets have just been released for Andy Borowitz's New Yorker Festival show this Friday night in New York City. Buy tickets here.", "")
    article = article.replace("(Satire from The Borowitz Report)", "")
    article = article.replace("Tickets for Andy Borowitz's next live show are now on sale.  Photograph by Alex Wong/Getty.", "")
    article = article.replace("Tickets for Andy Borowitz's next live show are now on sale.      Illustration by Tom Bachtell.", "")
    article = article.replace("Andy Borowitz will be doing two shows at next month's New Yorker Festival: Friday, October 5th, with the storytelling group The Moth, and Saturday, October 6th, with Sarah Silverman. Ticket information here.    Photograph by Tony Avelar/Bloomberg/Getty Images.", "")
    article = article.replace("(The Borowitz Report)", "")
    article = article.replace("Tickets for Andy Borowitz's next live show are now on sale.  Photograph by Chris Maddaloni/CQ Roll Call.", "")
    article = article.replace("(Satire from The Borowitz Report)", "")
    article = article.replace('"', " ").replace("'",'').replace('“',' ').replace('”',' ').replace("’",'').replace('-',' ').replace('--',' ').replace('—',' ').replace('…',' ')
    article = "".join((char.lower() for char in article if char not in string.punctuation))
    return article

In [3]:
# This function scrapes the article text, article title, and article date for an article
def text_scraper(link):
    url = 'https://www.newyorker.com'+link
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    article_text = soup.find("div", attrs={"id":"articleBody"}).text
    article_text = article_clean(article_text)
    title = soup.find('title').text
    date = soup.find('p', attrs={'class':"ArticleTimestamp__timestamp___1klks "}).text
    return [title, date, url, article_text]

In [4]:
text = text_scraper('/humor/borowitz-report/used-tool-purchased-1-8-million')
text

['Used Tool Purchased for $1.8 Million | The New Yorker',
 'September 3, 2014',
 'https://www.newyorker.com/humor/borowitz-report/used-tool-purchased-1-8-million',
 'new york  a used tool that has been on the market for several months was purchased on tuesday for the whopping price of 18 millionwhile it may not have set a new record the sale price raised eyebrows with many observers noting that 18 million was a lot to pay for this particular toolbut others defended the purchase arguing that the tool had been successfully bought and sold many times in the past']

In [19]:
# Store what text_scraper returns in d
d = []

# If a hiccup occurs (i.e., if a request was blocked), store the link that caused it in not_scraped
not_scraped = []

for link in links:
    print('Scraping stuff from link %s' %link)
    sleep(2)
    try:
        d.append(text_scraper(link))
    except:
        not_scraped.append(link)
        continue

# If hiccups occurred, go back and try to scrape again
if not_scraped:
    for link in not_scraped:
        print('Scraping stuff from link %s' %link)
        sleep(2)
        d.append(text_scraper(link))

Scraping stuff from link /humor/borowitz-report/pence-calls-space-force-necessary-to-protect-us-from-gay-aliens
Scraping stuff from link /humor/borowitz-report/republicans-projected-to-pick-up-seventy-seats-in-prison
Scraping stuff from link /humor/borowitz-report/republicans-outspent-democrats-in-ohio-by-a-hundred-million-rubles
Scraping stuff from link /humor/borowitz-report/californians-puzzled-by-trumps-failure-to-blame-wildfires-on-hillary
Scraping stuff from link /humor/borowitz-report/oliver-north-vows-to-raise-money-for-the-nra-by-selling-arms-to-iran
Scraping stuff from link /humor/borowitz-report/millions-of-americans-denied-groceries-after-failing-to-provide-id
Scraping stuff from link /humor/borowitz-report/trump-family-flees-to-moscow
Scraping stuff from link /humor/borowitz-report/republicans-accuse-rosenstein-of-secretly-plotting-to-uphold-constitution
Scraping stuff from link /humor/borowitz-report/ivanka-trump-named-dean-of-business-school-at-trump-university
Scraping 

Scraping stuff from link /humor/borowitz-report/kim-jong-un-taunts-trump-with-photo-of-hair-withstanding-gale-force-wind
Scraping stuff from link /humor/borowitz-report/military-refuses-to-participate-in-trumps-parade-citing-bone-spurs
Scraping stuff from link /humor/borowitz-report/under-pressure-from-fox-news-obama-to-stop-making-stocks-plummet
Scraping stuff from link /humor/borowitz-report/trump-considering-firing-dow-jones-industrial-average
Scraping stuff from link /humor/borowitz-report/former-hippies-put-in-horrible-position-of-rooting-for-fbi
Scraping stuff from link /humor/borowitz-report/trump-collapses-from-exhaustion-after-ninety-minutes-of-faking-empathy
Scraping stuff from link /humor/borowitz-report/sarah-huckabee-sanders-proofreads-state-of-the-union-to-remove-any-remaining-facts
Scraping stuff from link /humor/borowitz-report/fox-and-friends-putting-finishing-touches-on-trumps-state-of-the-union-address
Scraping stuff from link /humor/borowitz-report/video-emerges-of-

Scraping stuff from link /humor/borowitz-report/cbs-criticized-for-airing-graphic-horror-program-without-viewer-advisory
Scraping stuff from link /humor/borowitz-report/researchers-identify-gene-for-awfulness
Scraping stuff from link /humor/borowitz-report/eight-hundred-thousand-people-with-dreams-deported-by-one-with-delusions
Scraping stuff from link /humor/borowitz-report/obama-cruelly-taunted-trump-in-letter-riddled-with-multisyllabic-words
Scraping stuff from link /humor/borowitz-report/trumps-horrific-spelling-reassures-nation-that-he-cannot-correctly-enter-nuclear-codes
Scraping stuff from link /humor/borowitz-report/trump-says-sun-equally-to-blame-for-blocking-moon
Scraping stuff from link /humor/borowitz-report/controversy-over-confederate-statues-inspires-betsy-devos-to-google-civil-war
Scraping stuff from link /humor/borowitz-report/ivanka-and-jared-vacationing-in-moral-vacuum
Scraping stuff from link /humor/borowitz-report/americans-demand-that-offensive-symbol-of-racism-be

Scraping stuff from link /humor/borowitz-report/an-important-message-from-mike-pence
Scraping stuff from link /humor/borowitz-report/disturbed-man-gets-past-white-house-security-gives-press-conference
Scraping stuff from link /humor/borowitz-report/more-than-three-hundred-million-americans-now-enrolled-in-trump-university
Scraping stuff from link /humor/borowitz-report/moving-vans-arrive-at-white-house-to-remove-all-traces-of-competence-dignity
Scraping stuff from link /humor/borowitz-report/bush-counting-down-days-until-he-is-no-longer-worst-president-in-history
Scraping stuff from link /humor/borowitz-report/karaoke-machine-backs-out-of-performing-at-inauguration
Scraping stuff from link /humor/borowitz-report/russia-bans-meryl-streep-movies
Scraping stuff from link /humor/borowitz-report/trump-urges-spy-agencies-to-lay-off-russia-and-focus-on-threat-posed-by-actresses
Scraping stuff from link /humor/borowitz-report/intel-chiefs-say-trumps-twitter-account-was-hacked-by-four-year-old


Scraping stuff from link /humor/borowitz-report/poll-americans-worried-about-keeping-families-safe-from-rudy-giuliani
Scraping stuff from link /humor/borowitz-report/ben-carson-confident-that-trump-will-choose-him-as-v-p
Scraping stuff from link /humor/borowitz-report/cut-from-g-o-p-convention-tebow-hopes-to-catch-on-with-other-political-party
Scraping stuff from link /humor/borowitz-report/furious-christie-refuses-to-pick-up-trumps-dry-cleaning
Scraping stuff from link /humor/borowitz-report/congressional-republicans-vote-to-abolish-f-b-i
Scraping stuff from link /humor/borowitz-report/trumps-bid-to-become-born-again-fails-as-jesus-turns-down-friend-request
Scraping stuff from link /humor/borowitz-report/british-lose-right-to-claim-that-americans-are-dumber
Scraping stuff from link /humor/borowitz-report/cash-strapped-trump-campaign-auctions-chris-christie-on-ebay
Scraping stuff from link /humor/borowitz-report/lebron-feeds-entire-city-of-cleveland-with-loaves-and-fishes
Scraping stuf

...

Scraping stuff from link /humor/borowitz-report/g-o-p-rivals-question-rubios-ignorance-credentials
Scraping stuff from link /humor/borowitz-report/poll-millions-of-americans-who-need-jobs-want-congress-to-get-to-bottom-of-this-benghazi-thing-first
Scraping stuff from link /humor/borowitz-report/in-landmark-decision-supreme-court-strikes-down-main-reason-country-was-started
Scraping stuff from link /humor/borowitz-report/millionaires-unite-to-defeat-minimum-wage
Scraping stuff from link /humor/borowitz-report/sterling-says-he-will-miss-being-around-people-he-hates
Scraping stuff from link /humor/borowitz-report/donald-sterling-apologizes
Scraping stuff from link /humor/borowitz-report/republicans-blast-nevada-rancher-for-failing-to-use-commonly-accepted-racial-code-words
Scraping stuff from link /humor/borowitz-report/georgia-unveils-new-slogan-we-make-florida-look-safe
Scraping stuff from link /humor/borowitz-report/supreme-court-calls-lying-by-politicians-an-expression-of-their-religi

Scraping stuff from link /humor/borowitz-report/cruz-plans-to-read-affordable-care-act
Scraping stuff from link /humor/borowitz-report/obama-declares-national-day-of-gloating
Scraping stuff from link /humor/borowitz-report/cruz-the-dream-of-keeping-poor-people-from-seeing-a-doctor-must-never-die
Scraping stuff from link /humor/borowitz-report/rand-paul-proposes-reopening-just-enough-of-government-to-hold-new-hearings-on-benghazi
Scraping stuff from link /humor/borowitz-report/poll-americans-divided-over-what-wild-animal-they-would-like-to-see-congress-mauled-by
Scraping stuff from link /humor/borowitz-report/hostage-takers-call-comparisons-to-tea-party-hurtful
Scraping stuff from link /humor/borowitz-report/republicans-shut-down-prefrontal-cortex
Scraping stuff from link /humor/borowitz-report/ted-cruz-destroying-entire-planet-best-way-to-stop-obamacare
Scraping stuff from link /humor/borowitz-report/boehner-obama-stubbornly-refusing-to-end-crisis-i-created
Scraping stuff from link /hu

Scraping stuff from link /humor/borowitz-report/bitter-scalia-leaves-u-s
Scraping stuff from link /humor/borowitz-report/scalia-furious-he-has-to-hear-about-gay-couples-all-week
Scraping stuff from link /humor/borowitz-report/scalia-says-marriage-views-not-affected-by-lifelong-fear-of-gays
Scraping stuff from link /humor/borowitz-report/n-r-a-cant-believe-how-well-national-conversation-about-guns-is-going
Scraping stuff from link /humor/borowitz-report/cheney-marks-tenth-anniversary-of-pretending-there-was-reason-to-invade-iraq
Scraping stuff from link /humor/borowitz-report/portman-inspires-other-republicans-to-stop-speaking-to-their-children
Scraping stuff from link /humor/borowitz-report/unsuccessful-pope-candidate-blames-media
Scraping stuff from link /humor/borowitz-report/vatican-calls-white-smoke-a-false-alarm-we-were-just-burning-documents
Scraping stuff from link /humor/borowitz-report/ryan-budget-plan-gets-endorsement-from-ayn-rand
Scraping stuff from link /humor/borowitz-rep

Scraping stuff from link /humor/borowitz-report/poll-race-close-among-voters-with-no-television
Scraping stuff from link /humor/borowitz-report/clinton-regrets-reading-short-version-of-speech
Scraping stuff from link /humor/borowitz-report/democrats-revise-convention-schedule
Scraping stuff from link /humor/borowitz-report/ryan-gives-detailed-account-of-day-he-killed-bin-laden
Scraping stuff from link /humor/borowitz-report/biden-still-waiting-to-be-told-where-convention-is
Scraping stuff from link /humor/borowitz-report/ryan-calls-nomination-greatest-triumph-since-winning-tour-de-france
Scraping stuff from link /humor/borowitz-report/poll-romney-trails-empty-chair
Scraping stuff from link /humor/borowitz-report/romney-accepts-nomination-i-bought-it
Scraping stuff from link /humor/borowitz-report/ryan-launches-campaign-theme-of-lying-about-everything
Scraping stuff from link /humor/borowitz-report/disturbed-man-gets-past-convention-security-gives-keynote-address
Scraping stuff from lin

Scraping stuff from link /humor/borowitz-report/an-argument-against-health-care-from-the-national-alliance-of-funeral-directors
Scraping stuff from link /humor/borowitz-report/a-letter-from-kim-jong-un-2
Scraping stuff from link /humor/borowitz-report/a-farewell-from-herman-cain
Scraping stuff from link /humor/borowitz-report/potential-matchup-between-black-man-and-mormon-poses-dilemma-for-bigots
Scraping stuff from link /humor/borowitz-report/obamas-use-of-complete-sentences-stirs-controversy
Scraping stuff from link /humor/borowitz-report/a-clarification-from-mitt-romney-about-poor-people
Scraping stuff from link /humor/borowitz-report/republicans-reveal-that-entire-presidential-race-was-a-prank
Scraping stuff from link /humor/borowitz-report/hookers-downgrade-u-s-credit-rating
Scraping stuff from link /humor/borowitz-report/cheney-receives-heart-transplant-bush-still-on-waiting-list-for-brain
Scraping stuff from link /humor/borowitz-report/a-letter-from-mark-zuckerberg


In [20]:
with open('final_borowitz.json', 'w') as fp:
    json.dump(d, fp)

fp.close()